# Librerías.

In [44]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from PIL import Image
import torchvision.transforms as transforms
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from pathlib import Path
from PIL import Image, ImageDraw
from datetime import datetime
import json
from transformers import AutoFeatureExtractor, AutoModelForObjectDetection

import torchvision
import os
import pytorch_lightning as pl
import torch

import cv2
from pytorch_lightning import Trainer
from PIL import ImageFont
from transformers import YolosForObjectDetection, YolosImageProcessor


In [3]:
import yaml

# CONSTANTES.

In [4]:
ROOT_PATH="RDD2022/Img"

# Proceso de entrenamiento.

Se realizaron dos implementaciones de Fine-Tunning:

1. Modelo "hustvl/yolos-small" con 20 epochs utilizando las imagenes de Japón, Índia y Noruega.
2. Modelo "yolov8s.pt" con 100 epochs utilizando todas las imagenes disponibles en el conjunto de datos de entrenamiento.

## Modelo "yolov8s.pt" 

### 1. Obtención de data a un dataframe.

Se realizó un desarrollo para:

1. Obtener todas las posibles rutas a las imagenes disponibles.
2. El conjunto de datos, posee una carpeta con las imágenes y otra carpeta donde están las anotaciones de interés, por lo que se realizó una función para, dado un archivo .xml, obtener la clase, las coordenadas del bounding box (x1, y1, x2, y2), los valores de resolución de la imagen (Width, Height).

#### 1.1. Obtener todas las posibles rutas.

Al descomprimir el .zip de 12 GBs nombrado como RDD2022_released_through_CRDDC2022.zip y descomprimirlo, obtendrás carpetas repartidas por el país en que fue tomada la imagen, por lo que la siguiente función es para obtener la ruta absoluta desde la ruta raíz en que fue descomprimido la carpeta. En este caso, el comprimido fue extráido dentro de la carpeta RDD2022/Img, por lo que `root_path="RDD2022/Img"`

In [5]:
def get_paths_files(root_path, constraint="", search_img=""):
    """
        ** root_path:  Nombre del folder donde estan todas las carpetas por pais del comprimido dado por RDD2022_released_through_CRDDC2022.zip
        ** constraint: Filtrar la lista de todos los path dependiendo lo que contenga cada valor de la lista.
            Ej:
                get_paths_files(root_path="Img", constraint="train"), te dara todas las rutas donde aparezca,
                la palabra train.

    """
    #Ruta raiz de la carpeta de paises
    path_folder_country=root_path

    #Lista total de paths con imagenes y anotaciones
    l=[root_path+"/"+ctry_name+"/"+folder_cont_tra_tes+"/"+annot_imgs+"/"+value_annot_imgs+"/"
       #+value_annots
        for ctry_name in os.listdir(path_folder_country)
            for folder_cont_tra_tes in os.listdir(path_folder_country+"/"+ctry_name)
                for annot_imgs in os.listdir(path_folder_country+"/"+ctry_name+"/"+folder_cont_tra_tes)
                    for value_annot_imgs in os.listdir(path_folder_country+"/"+ctry_name+"/"+folder_cont_tra_tes+"/"+annot_imgs)
                        #for value_annots in os.listdir(path_folder_country+"/"+ctry_name+"/"+folder_cont_tra_tes+"/"+annot_imgs+"/"+value_annot_imgs)
                        ]
    # Filtrar por palabra clave
    if(len(constraint)!=0):
        r = filter(lambda p: constraint in p,l)
        l=list(r)

    # Filtrar por nombre de la imagen de la carpeta train
    if(len(search_img)!=0):
        r = filter(lambda p: search_img in p,l)
        l=list(r)

    return l

get_paths_files(ROOT_PATH)[0:3]

['RDD2022/Img/China_Drone/train/annotations/xmls/',
 'RDD2022/Img/China_Drone/train/images/China_Drone_000000.jpg/',
 'RDD2022/Img/China_Drone/train/images/China_Drone_000001.jpg/']

#### 1.2. Obtener todas las rutas a los archivos .xml.

Para obtener la ruta de los xml, se utilizó la función anterior, junto con una configuración de esta que permite filtrar por los valores de la lista, esto se logra cambiando el parámetro *constraint*, haciendo `get_paths_files(ROOT_PATH,constraint="xml")`

In [6]:
def get_xml_files():
    file_paths_xml=get_paths_files(ROOT_PATH,"xml")
    return [folder_xml+file_xml for folder_xml in file_paths_xml for file_xml in os.listdir(folder_xml)]

get_xml_files()[0:3]

['RDD2022/Img/China_Drone/train/annotations/xmls/China_Drone_000000.xml',
 'RDD2022/Img/China_Drone/train/annotations/xmls/China_Drone_000001.xml',
 'RDD2022/Img/China_Drone/train/annotations/xmls/China_Drone_000002.xml']

#### 1.3. Obtener los datos de las anotaciones de cada imagen.

Para obtener los valores usando los encabezados de los archivos .xml, se utilizó la siguiente función apoyandose de la librería ` xml.etree.ElementTree`, que convierte la estructura del archivo en un árbol que posee clave: valor, para poder acceder a este a través de la jerarquía de los encabezados.

Los valores que devolverá serán de la siguiente forma en una tupla:

```
('RDD2022/Img/China_Drone/train/images/China_Drone_000001.jpg',
 ['D10', 'D00'],
 [[0.20703125, 0.1650390625, 0.359375, 0.056640625],
  [0.2099609375, 0.376953125, 0.048828125, 0.30859375]],
 512,
 512)
```

Notemos que tanto las clases como los bounding boxes normalizados están en listas y que cada índice estas, corresponden entre sí, es decir: El bounding box asociado a D10 es [0.20703125, 0.1650390625, 0.359375, 0.056640625] y para D00 es [0.2099609375, 0.376953125, 0.048828125, 0.30859375]. Se decidió esta estructura debido a que se asemeja a las usadas al COCO dataset benchmark.

In [7]:
# Funcion para obtener el nombre del archivo sin la extensión.
def get_name_file(path):
    l=path.split("/")
    return l[len(l)-1].split(".")[0]

In [8]:
def get_meta_data_xml(path_xml):
    tree = ET.parse(path_xml)
    # Obtener el arbol
    root = tree.getroot()

    # list_tuples=[]

    # Obtener size
    W = int(root.find("size").find("width").text)
    H = int(root.find("size").find("height").text)

    list_labels=[]
    list_bboxes=[]
    # Por cada objeto encontrado

    #Anadir la imagen a la lista de imagenes
    name_file=get_name_file(path_xml)
    path_img=get_paths_files(ROOT_PATH,"images",name_file)[0]
    
    for obj in root.findall("object"):

        #Obtener los datos necesarios: label, y para armar la caja
        label=obj.find("name").text
        list_labels.append(label)

        xmin=float(obj.find("bndbox").find("xmin").text)
        ymin=float(obj.find("bndbox").find("ymin").text)
        xmax=float(obj.find("bndbox").find("xmax").text)
        ymax=float(obj.find("bndbox").find("ymax").text)

        # Convertir a (x, y, w, h)
        x = (xmin + xmax) / 2.0
        y = (ymin + ymax) / 2.0
        w = xmax - xmin
        h = ymax - ymin

        # Normalizar
        x_norm = x / W
        y_norm = y / H
        w_norm = w / W
        h_norm = h / H

        list_bboxes.append([x_norm, y_norm, w_norm, h_norm])

    return (path_img[0:len(path_img)-1],list_labels,list_bboxes, W, H)

xml_aux=get_xml_files()[0:3]
get_meta_data_xml(xml_aux[1])

('RDD2022/Img/China_Drone/train/images/China_Drone_000001.jpg',
 ['D10', 'D00'],
 [[0.20703125, 0.1650390625, 0.359375, 0.056640625],
  [0.2099609375, 0.376953125, 0.048828125, 0.30859375]],
 512,
 512)

#### 1.4. Creación del dataframe.

El siguiente código es costoso debido a la cantidad de imágenes totales involucradas. Por cada .xml, se realiza un registro a un dataframe con columnas `"Path","L_Label","L_Bbox","Width","Height"`, llegando a un total de 38384, de los cuáles, eliminando registros que no poseían Label ó bounding box, terminaría siendo 26 mil imágenes aproximadamente para usarse. 

Tardó cerca de 1 hora en procesar las 38 mil imágenes e ingresar los registros en un dataframe. Luego de eliminar aquellas imagenes que no tenían clasificación, se guardó el dataframe en un archivo `df_acum_notEmpyBbox.xlsx`, por lo que se convirtió en un checkpoint para el desarrollo posterior, teniendo que cargar dicho archivo en lugar de correr este código.

```python
# Inicializacion de dataframe
df_metadata_acum_2=pd.DataFrame(columns=["Path","L_Label","L_Bbox","Width","Height"])

# Obtener todas las rutas de los elementos xml que contiene labels y coordenadas
xml_paths=get_xml_files()

#Contador para ver si se esta agregando
con=0

# Por cada ruta del arxhico xml
for xml_path in xml_paths:
    # print(xml_path)
    # Obtener sus metadatos por label y coordenadas
    list_rows=get_meta_data_xml(xml_path)
    
    df_metadata_acum_2.loc[len(df_metadata_acum_2.index)] = list_rows

    # Verificando con los primeros 100 paths si se esta cargando bien

    #if(con==100):
        #print(list_rows)
    #    break
    #con+=1

df_metadata_acum_2.tail(4)
```

In [9]:
# Cargar dataframe de las imagenes que tienen alguna clasificacion

df_acum_ALL_notEmpty=pd.read_excel('df_acum_notEmpyBbox.xlsx')
df_acum_ALL_notEmpty.drop(["Unnamed: 0"],axis=1,inplace=True)

In [10]:
df_acum_ALL_notEmpty.tail(4)

,Path,L_Label,L_Bbox,Width,Height
26657,Img/United_States/train/images/United_States_0...,['D10'],"[[0.36171875, 0.78125, 0.0859375, 0.01875]]",640,640
26658,Img/United_States/train/images/United_States_0...,"['D00', 'D00']","[[0.784375, 0.859375, 0.3, 0.26875], [0.808593...",640,640
26659,Img/United_States/train/images/United_States_0...,"['D00', 'D00']","[[0.51875, 0.925, 0.1, 0.146875], [0.65625, 0....",640,640
26660,Img/United_States/train/images/United_States_0...,['D00'],"[[0.13125, 0.69375, 0.2625, 0.159375]]",640,640


### 2. Creación de carpetas con estructura requerida.

Basado en el ejemplo de Fine-Tunning de la detección de basura del mar (https://developer.ibm.com/tutorials/awb-train-yolo-object-detection-model-in-python/), indica se se debe tener una carpeta "datasets" que contenga en su raíz, en este caso para entrenamiento:

```
datasetsFolder
    -- Train
        -- images
            -- image1.jpg
            -- image2.jpg
            ...
        -- labels
            -- image1.txt
            -- image2.txt
            ...
```

En los archivos txt, deben aparecer la información del label codificado y posteriormente los valores x, y, w, h estandarizados con respecto a la imagen original. Por ejemplo, este sería el archivo para `China_Drone_000003.txt`:

```txt
0 0.2080078125 0.93359375 0.380859375 0.04296875
0 0.6943359375 0.638671875 0.611328125 0.08984375
1 0.46484375 0.291015625 0.15234375 0.578125
```
Para lograr esa estructura de archivos y carpetas, se realizó la siguiente implementación.

#### 2.1. Codificación de Labels.

Algunos datos son necesarios para realizar un correcto Fine-tunning para el modelo YOLOS, en este caso, está la codificación de las clases, para ello, se realizó el siguiente código para conocer cuáles eran las posibles clases diferentes.

In [11]:
# Todos los posibles valores de label

# Lista de valores uniquos
label_uniques=[]

# Por cada fila del dataframe

for i in range(0,len(df_acum_ALL_notEmpty)):
    row=df_acum_ALL_notEmpty.iloc[i]
    #Obtener la lista
    list_labels=eval(row["L_Label"])
    for label in list_labels:
        if(label not in label_uniques):
            label_uniques.append(label)
        else:
            break


En este caso, hubo un error de preprocesamiento que posteriormente no se cambió y es la presencia de la clase `D0w0` en una única imagen, dicha clase sería `D00`. Sin embargo, observaremos que no afecto de manera significativa el entrenamiento del modelo.

In [12]:
# Crear diccionario con todos los posibles valores
label2Value={i: label_uniques[i] for i in range(0,len(label_uniques))}
value2Label={v: k for k,v in label2Value.items()}
label2Value

{0: 'D10',
 1: 'D00',
 2: 'D20',
 3: 'Repair',
 4: 'D40',
 5: 'Block crack',
 6: 'D44',
 7: 'D01',
 8: 'D11',
 9: 'D43',
 10: 'D50',
 11: 'D0w0'}

#### 2.2. Configuración de archivo .yaml y convenciones.

El punto de enlace entre las rutas de los archivos de entrenamiento y la configuración del momento de entrenamiento, es por medio de un archivo nombrado de la siguiente forma: `config.yaml`. En dicho archivo se especifican las rutas donde están las imagenes de train, test y val, en este caso, todas las imagenes de entrenamiento se usarán, por lo que, así se establezca en la configuración, no habrá imagenes en test y val.

El contenido del archivo se crea con la siguiente implementación.

In [13]:
# Establecer una ruta raiz

# store working directory path as work_dir
work_dir = "."

# print work_dir path
print(os.getcwd())

# print work_dir contents
print(os.listdir(f"{work_dir}"))

d:\Documents\Universidad Nacional de Colombia\2024 - 1\3009150 Redes Neuronales Artificiales y Algoritmos Bio-Inspirados\TFinal\E2\CRDDC\metricasIoUdataJapan
['args.yaml', 'datasets', 'detr', 'df_acum_notEmpyBbox.xlsx', 'df_metadata_acum_ALL.xlsx', 'df_metada_acum.xlsx', 'functionalDataYolo.ipynb', 'functionalHealthyRoad.ipynb', 'imagen_con_bbboxes.jpg', 'metadata_damage.xlsx', 'metricasIoUdataJapan.png', 'metricasIoUdataJin.png', 'New folder', 'pielCocoDriloCarretera.jpeg', 'RDD2022', 'results.csv', 'roaddamages', 'runs', 'save_japan', 'save_jin', 'trainingHealthyRoad.ipynb', 'United_States_004802.txt', 'util.py', 'vias-dañadas.jpeg', 'weights', 'yolov8n.pt', 'yolov8s.pt', 'yolov8_DAMT.ipynb', '__pycache__']


In [14]:
# contents of new confg.yaml file
def update_yaml_file(file_path):
    data = {
        'path': f'{work_dir}/roaddamages/data_test_coco',
        'train': 'train/images',
        'val': 'train/images',
        'test': 'test/images',
        'names': label2Value
    }

    # ensures the "names" list appears after the sub/directories
    names_data = data.pop('names')
    with open(file_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file)
        yaml_file.write('\n')
        yaml.dump({'names': names_data}, yaml_file) 

if __name__ == "__main__":
    file_path = f"{work_dir}/roaddamages/data_test_coco/config.yaml" #.yaml file path
    update_yaml_file(file_path)
    print(f"{file_path} updated successfully.")

./roaddamages/data_test_coco/config.yaml updated successfully.


##### **Convenciones**.

El archivo `config.yaml` creado, en este caso en particular, tiene el siguiente contenido.

```yaml
path: ./roaddamages/data_test_coco
test: test/images
train: train/images
val: train/images

names:
  0: D10
  1: D00
  2: D20
  3: Repair
  4: D40
  5: Block crack
  6: D44
  7: D01
  8: D11
  9: D43
  10: D50
  11: D0w0
```

Algo que **destacar**, es que, por fuera de la carpeta **roaddamages**, debe llamarse **datasets** también, esto es por una configuración en la API a la hora de realizar el entrenamiento y buscar las imágenes. Dejando la estructura de la carpepta de la siguiente manera.

```txt
datasets                    (Obligatorio)
  -- roaddamages            (Arbitrario)
    -- data_test_coco       (Arbitrario)
      -- train              (Obligatorio)
        -- images           (Obligatorio)
            -- image1.jpg
            -- image2.jpg
            ...
        -- labels           (Obligatorio)
            -- image1.txt
            -- image2.txt
            ...

```

#### 2.3. Creación de archivos .txt

Teniendo la estructura de carpetas creada, se procede a llenar la carpeta `labels` con `.txt` que tienen como nombre la imagen correspondiente y que cuenta con la información de `label_codificado x y w h` de cada detección de objeto separado por enter.

Para ello, se creó la siguiente implementación que toma los registros del dataframe creado en **1.**, y con dicha información, escribe el contenido correspondiente.

```python
# Por cada fila del dataframe

for index_row in range(0,len(df_acum_ALL_notEmpty)):
    # Obtener la fila
    row=df_acum_ALL_notEmpty.iloc[index_row]

    # Nombre txt
    name_txt=get_name_file(row["Path"])+".txt"

    # Por cada label, hay un bbox
    labels=eval(row["L_Label"])
    bboxes=eval(row["L_Bbox"])

    # cadena de texto auxiliar
    str_aux=""
    for i in range(0,len(labels)):
        bbox=bboxes[i]
        x=bbox[0]
        y=bbox[1]
        w=bbox[2]
        h=bbox[3]

        lab_enc=value2Label[labels[i]]
        if(i==len(labels)-1):
                str_aux=str_aux+f"{lab_enc} {x} {y} {w} {h}"
        else:
            str_aux=str_aux+f"{lab_enc} {x} {y} {w} {h}\n"

    # Crear y escribir en el archivo
    with open("./roaddamages/data_test_coco/train/labels/"+name_txt, 'w') as archivo:
        archivo.write(f'{str_aux}')  # Añadir cada valor en una nueva línea

```

#### 2.4. Pasar imagenes a carpeta `images`.

Teniendo presente que al descomprimir las imágenes estas son guardadas por carpetas por país, se realizará una implementación para tomar cada ruta de la imagen, usando el dataframe creado en **1.** y moverla a la carpeta de `train/images`.

```python
# Pasar las imagenes con labels asociados
for index_row in range(0,len(df_acum_ALL_notEmpty)):

    # Con una imagen
    row=df_acum_ALL_notEmpty.iloc[index_row]

    # Path de la imagen origen: Son de la forma "Img/United_States/train/images/United_States_004801.jpg"
    path_origen="./RDD2022/"+row["Path"]

    # Nombre de la imagen
    name_jpg=get_name_file(row["Path"])+".jpg"

    # Path destino
    path_destino="./roaddamages/data_test_coco/train/images/"+name_jpg

    # Copiar la imagen de la carpeta original a la carpeta destino
    shutil.copy(path_origen, path_destino)
```

### 3. Entrenamiento.

Basado en el ejemplo de detección de basura en el mar [https://developer.ibm.com/tutorials/awb-train-yolo-object-detection-model-in-python/], se muestra que para realizar el entrenamiento se utiliza el siguiente comando.

```cmd
!yolo task=detect mode=train data={work_dir}/roaddamages/data_test_coco/config.yaml model=yolov8s.pt epochs=100 batch=32 lr0=.4 plots=True device=0
```

Al acabar la ejecución, en la carpeta `runs/detect` se tendrá el comportamiento del modelo final con diferentes métricas: Curva ROC, IoU, Precision, Recall, entre otras.

In [22]:
!yolo task=detect mode=train data={work_dir}/roaddamages/data_test_coco/config.yaml model=yolov8s.pt epochs=100 batch=32 lr0=.4 plots=True device=0

^C


#### 3.1. Resultados del entrenamiento

Al acabar la ejecución del entrenamiento, el último modelo guardado quedó en la carpeta `runs/detect/train25/weights/best.pt`. Además, se generaron un conjunto de resultados asociados a la métricas del entrenamiento realizado. 

Cabe resaltar que las clases están desbalanceadas, por lo que algunas métricas tenderán a inflarse.

#### 3.1.1. Resumen de métricas

<img src="./train25/metricasV8pt100epocs.png">



##### Conclusiones.

* En un 75.7% aproximadamente las bounding box son verdaderas en comparación a las coordenadas reales. Aunque, para la clase D11, el valor está cerca de ser del 50% [IEEE Standards Association. (2021). IEEE Standard for Model Process for Addressing Ethical Concerns During System Design (IEEE Std 7000-2021). doi: 10.1109/IEEESTD.2021.9442728.].

* En un 63.4% el modelo detecta los objetos de interés del total de todas las instancias. Además, al tener mayor precisión que recall, muestra que, a la hora de predecir, el modelo prefiere ser preciso a obtener un falso positivo, lo que puede provocar que se están perdiendo de vista, algunos objetos de interés [IEEE Standards Association. (2021). IEEE Standard for Model Process for Addressing Ethical Concerns During System Design (IEEE Std 7000-2021). doi: 10.1109/IEEESTD.2021.9442728.].

* Considerando el IoU por debajo del 50% (Que para considerar un bounding box predicho como válido, debe intersectar como mínimo el 50% con el bounding box real) se obtiene un 73.1% en el balance entre Precision-Recall con este umbral, lo que sugiere que el modelo es capaz de localizar e identificar la mayoría de objetos de una imagen. Además, la mayoría de clases está por encima del valor promedio, a pesar del desbalance de instancias [Everingham, M., Eslami, S. M. A., Van Gool, L., Williams, C. K. I., Winn, J., & Zisserman, A. (2015). The Pascal Visual Object Classes Challenge: A Retrospective. International Journal of Computer Vision, 111(1), 98-136. doi: 10.1007/s11263-014-0733-5.].

* Considerando el IoU con umbrales desde 0.5 hasta 0.95 en incrementos de 0.05, se obtiene un valor promedio de 0.49. Al ser una medida más estricta indica que el 49% de las detecciones realizadas son correctas considerando distintos niveles de umbral IoU siento la clase `Repair` la que más sobresale seguido de D43. Esto muestra, que si bien el modelo es sólido para la detección, a niveles altos de precisión, requiere de mayor ajuste o el margen de error es de casi 50% [Lin, T. Y., Maire, M., Belongie, S., Hays, J., Perona, P., Ramanan, D., Dollár, P., & Zitnick, C. L. (2014). Microsoft COCO: Common Objects in Context. In European Conference on Computer Vision (ECCV), 2014, 740-755. doi: 10.1007/978-3-319-10602-1_48.].



##### 3.1.2. Matriz de confusión normalizada.

<img src="./train25/confusion_matrix_normalized.png">

##### Conclusiones.

* Existe una clase más, llamada `background` que hace referencia a una clasificación que no correspone a ninguno de los tipos de daños en carretera. Con esto, notamos que la clase D10 y D00 (Las clases que más concurrencia tienen), tienen problemas para clasificarse con cierto tipo de `background`.

* La clase no procesada `D0w0` se clasificó como D20, cuando debió ser clasificada como D00.

* Cerca del 60% de los datos de `Block crack` y `Repair` no fueron bien clasificados.

* Los daños D10, D00, D20, D40, Block crack, D01 y D11, muestran que entre un 20 a 40% de sus clasificaciones son confundidas con el fondo de la imagen, lo que indica que algunas de estas imágenes presentan mucho ruído que afecta el entrenamiento del modelo.

## Modelo hustvl/yolos-small.

Para realizar la implementación de Fine-Tunning de este modelo, se siguió el ejemplo de [https://github.com/NielsRogge/Transformers-Tutorials/blob/master/YOLOS/Fine_tuning_YOLOS_for_object_detection_on_custom_dataset_(balloon).ipynb], donde desean clasificar si en la imagen hay balones o no.

Para dejar los datos a punto se deben crear jsons tanto de entrenamiento como de validación siguiendo el formato COCO. 

### 1. Creación de dataframe y disposición de datos.

Para la creación del dataframe se utilizaron las mismas secuencia de implementaciones que en ítem **1. del modelo v8**. Sin embargo, los bounding box fueron dejados con los valores iniciales del .xml y se dejó un valor dummie para el Width y Height de la imagen, ya que, en la clase que se usará para cargar los datos, toman los tamaños originales de la imagen.

El dataframe resultante posee las columnas `Path, Labels, Bboxes y TiposDanios`. Donde por cada clase hay un bounding box y un tipo de daño. Los tipos de daños son clasificaciones más generales de las clases. La anterior información es la que se desea mostrar de cara al cliente, ya que el interés principal de la aplicación es mostrar el tipo de daño general encontrado en la carretera (Esto aplica para el anterior modelo). 

Si bien tanto este modelo como el anterior fue entrado por la clasificación técnica del daño, a la hora de mostrar los resultados, se realizará por el tipo general. 

Usando las imágenes que poseen información de daños, se obtuvieron 25 mil registros aproximadamente, y las clases se redujeron, ya que se utilizarán las de mayor concurrencia.

In [15]:
# Path en caso de estar en google drive
google_drive=False
p_metadata_3="df_metada_acum.xlsx"
df_metadata_acum=pd.read_excel(p_metadata_3)
df_metadata_acum.drop(["Unnamed: 0"],axis=1,inplace=True)
df_metadata_acum.tail(4)

,Path,Labels,Bboxes,TiposDanios
25722,Img/United_States/train/images/United_States_0...,['D10'],"[[204.0, 494.0, 55.0, 12.0]]",['lateral linear crack']
25723,Img/United_States/train/images/United_States_0...,"['D00', 'D00']","[[406.0, 464.0, 192.0, 172.0], [410.0, 401.0, ...","['logitudinal linear crack', 'logitudinal line..."
25724,Img/United_States/train/images/United_States_0...,"['D00', 'D00']","[[300.0, 545.0, 64.0, 94.0], [356.0, 516.0, 12...","['logitudinal linear crack', 'logitudinal line..."
25725,Img/United_States/train/images/United_States_0...,['D00'],"[[0.0, 393.0, 168.0, 102.0]]",['logitudinal linear crack']


In [16]:
# Recordemos que en este caso particular, la carpeta segmentada por paises está ubicada en RDD2022/Img

df_metadata_acum["Path"]=list(map(lambda x: "RDD2022/"+x,list(df_metadata_acum["Path"])))

In [17]:
df_metadata_acum.tail(4)

,Path,Labels,Bboxes,TiposDanios
25722,RDD2022/Img/United_States/train/images/United_...,['D10'],"[[204.0, 494.0, 55.0, 12.0]]",['lateral linear crack']
25723,RDD2022/Img/United_States/train/images/United_...,"['D00', 'D00']","[[406.0, 464.0, 192.0, 172.0], [410.0, 401.0, ...","['logitudinal linear crack', 'logitudinal line..."
25724,RDD2022/Img/United_States/train/images/United_...,"['D00', 'D00']","[[300.0, 545.0, 64.0, 94.0], [356.0, 516.0, 12...","['logitudinal linear crack', 'logitudinal line..."
25725,RDD2022/Img/United_States/train/images/United_...,['D00'],"[[0.0, 393.0, 168.0, 102.0]]",['logitudinal linear crack']


#### 1.1. Separación de datos en Train y Val.

En este caso, es requerido para este modelo tener un conjunto de datos de entrenamiento y un junto de datos de prueba. Para la construcción de este, se implementó la siguiente función que parte el conjunto de datos en un porcentaje arbitrario. La proporción de datos de entrenamiento y prueba, al ser desbalanceado, se optó por 90% de entrenamiento y 10% de pruebas, esto es, buscando minimizar el desbalanceo. Esto se podría mejorar, realizando una separación estratificado, pero debido a que la variable objetivo es una lista, se optó no usarlo por practicidad.

In [18]:
# Función para filtrar el dataframe por algun país en específico.

def get_df_bypath(df,constraint=""):
    df_filtered=df.copy()[df.copy()["Path"].str.contains(constraint)]
    return df_filtered.reset_index(drop=True)

In [19]:
# Función para separar el conjunto de datos de entrenamiento y prueba dado una proporción específica
def train_test_split_RDD(df, prop=0.2):
    X = df[["Path","Labels","Bboxes","TiposDanios"]]
    X_train, X_test = train_test_split(X, test_size=prop, random_state=18022022)
    return X_train, X_test

En este caso, y luego de una exploración de las imágenes, se observó que las imágenes de mejor calidad (donde hay menos ruido por paisajes) son las de Japón, además que es el país que más imágenes tiene con 9 mil aproximadamente. Por otra parte, las imágenes de India y Noruega, son las que mayor presentan ruído en sus imágenes y puede afectar el entrenamiento de manera negativa en caso de no incluirlas. Por esta razón, se decidió unir los dataframes de los países de Japón, India y China, y se realizó una separación de 10% de datos en prueba.

In [22]:
path_country="Japan"
df_j=get_df_bypath(df_metadata_acum,constraint=path_country)
df_j.tail(2)

,Path,Labels,Bboxes,TiposDanios
9309,RDD2022/Img/Japan/train/images/Japan_013131.jpg,['D20'],"[[475.0, 505.0, 90.0, 38.0]]",['alligator crack']
9310,RDD2022/Img/Japan/train/images/Japan_013132.jpg,"['D00', 'D43', 'D20']","[[542.0, 275.0, 58.0, 61.0], [189.0, 82.0, 221...","['logitudinal linear crack', 'other corruption..."


In [26]:
path_country="Norway"
df_norway=get_df_bypath(df_metadata_acum,constraint=path_country)
df_norway.tail(2)

,Path,Labels,Bboxes,TiposDanios
2912,RDD2022/Img/Norway/train/images/Norway_008156.jpg,"['D00', 'D00', 'D00', 'D00']","[[345.57, 1374.61, 720.8100000000002, 604.1000...","['logitudinal linear crack', 'logitudinal line..."
2913,RDD2022/Img/Norway/train/images/Norway_008159.jpg,"['D10', 'D00', 'D00']","[[1299.8, 1211.93, 229.04999999999995, 17.9299...","['lateral linear crack', 'logitudinal linear c..."


In [25]:
path_country="India"
df_india=get_df_bypath(df_metadata_acum,constraint=path_country)
df_india.tail(2)

,Path,Labels,Bboxes,TiposDanios
3769,RDD2022/Img/India/train/images/India_009889.jpg,['D43'],"[[102.0, 488.0, 421.0, 68.0]]",['other corruption']
3770,RDD2022/Img/India/train/images/India_009890.jpg,"['D40', 'D40']","[[238.0, 500.0, 56.0, 41.0], [301.0, 455.0, 56...","['other corruption', 'other corruption']"


In [29]:
# Union de los dataframes anteriores

# Concatenando por filas, ya que poseen las mismas columnas
df_tv = pd.concat([df_j,df_india,df_norway], axis=0).reset_index(drop=True)
df_tv.tail(2)

,Path,Labels,Bboxes,TiposDanios
15994,RDD2022/Img/Norway/train/images/Norway_008156.jpg,"['D00', 'D00', 'D00', 'D00']","[[345.57, 1374.61, 720.8100000000002, 604.1000...","['logitudinal linear crack', 'logitudinal line..."
15995,RDD2022/Img/Norway/train/images/Norway_008159.jpg,"['D10', 'D00', 'D00']","[[1299.8, 1211.93, 229.04999999999995, 17.9299...","['lateral linear crack', 'logitudinal linear c..."


In [30]:
X_train, X_test= train_test_split_RDD(df_tv,0.1)
print(len(df_tv), len(X_train), len(X_test))

15996 14396 1600


#### 1.2. Creación de jsons.

Usando el ejemplo del Fine-Tunning para clasificar balones, notamos que se crean jsons con una estructura específica. Es por esto, que se decidió crear los jsons a través de una implementación tendiendo presente la siguiente estructura, donde lso datos que están fijos, es porque así fueron tomados del ejemplo guía.

```python
# Obtener la fecha y hora actuales
fecha_actual = datetime.now()

# Formatear la fecha y hora al formato yyyy-mm-dd hh:mm:ss
fecha_formateada = fecha_actual.strftime("%Y-%m-%d %H:%M:%S")
```

```json
custom_json={
    "info": {
            "description": "Example Dataset", 
            "url": "https://github.com/waspinator/pycococreator", 
            "version": "0.1.0", 
            "year": 2018, 
            "contributor": "waspinator", 
            "date_created": f"{fecha_formateada}"
    },
    "licenses": [
            {
                "id": 1, 
                "name": "Attribution-NonCommercial-ShareAlike License", 
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
            }
    ],
    "categories": [
            {"id": 0, "name": "D10", "supercategory": "N/A"}, 
            {"id": 1, "name": "D00", "supercategory": "N/A"}, 
            {"id": 2, "name": "D20", "supercategory": "N/A"}, 
            {"id": 3, "name": "D40", "supercategory": "N/A"}, 
            {"id": 4, "name": "D44", "supercategory": "N/A"}, 
            {"id": 5, "name": "D01", "supercategory": "N/A"}, 
            {"id": 6, "name": "D11", "supercategory": "N/A"}, 
            {"id": 7, "name": "D43", "supercategory": "N/A"}
        ],
    "images": [
            {"id": 0, "file_name": "India/train/images/India_005399.jpg", "width": 600, "height": 600, "date_captured": "2024-09-05 16:16:08", "license": 1, "coco_url": "", "flickr_url": ""}, 
            {"id": 1, "file_name": "Japan/train/images/Japan_005085.jpg", "width": 600, "height": 600, "date_captured": "2024-09-05 16:16:08", "license": 1, "coco_url": "", "flickr_url": ""},
            ... 
        ],
    "annotations":[
        {"id": 0, "image_id": 0, "category_id": 2, "iscrowd": 0, "area": 0, "bbox": [173, 518, 547, 197], "segmentation":[[0]]}, 
        {"id": 1, "image_id": 0, "category_id": 3, "iscrowd": 0, "area": 0, "bbox": [541, 610, 82, 50], "segmentation": [[0]]}, {"id": 2, "image_id": 1, "category_id": 1, "iscrowd": 0, "area": 0, "bbox": [130, 356, 118, 103], "segmentation": [[0]]}, 
        {"id": 3, "image_id": 1, "category_id": 1, "iscrowd": 0, "area": 0, "bbox": [319, 336, 87, 95], "segmentation": [[0]]}, 
        ... 
        ]
}
```
Notemos que los registros se separan por id, tanto las imagenes como los bounding boxes. Para lograr dicho formato, se realiza la siguiente implementación, donde, al guardarse ambos archivos, se debe **tener presente la ruta raíz de este:** `RDD2022/Img/`. Esto es, porque a partir esa ruta raíz, se buscará la imagen para el entrenamiento, por lo que sino está en dicha ruta, saldrá un error en las rutas.

In [31]:
# Funcion para obtener nombre del archivo dado un path
def get_name_file(path):
    l=path.split("/")
    return l[len(l)-1].split(".")[0]

In [32]:
# Columna para asignar los valores de cada categoria
LABEL_MAPEADO={'D10':0, 'D00':1, 'D20':2,'D40':3, 'D44':4, 'D01':5,'D11':6, 'D43': 7}

In [34]:
LABELS=[k for k, v in LABEL_MAPEADO.items()]
LABELS

['D10', 'D00', 'D20', 'D40', 'D44', 'D01', 'D11', 'D43']

In [35]:
def create_json(df, save_json="", train=True):
    # Obtener la fecha y hora actuales
    fecha_actual = datetime.now()

    # Formatear la fecha y hora al formato yyyy-mm-dd hh:mm:ss
    fecha_formateada = fecha_actual.strftime("%Y-%m-%d %H:%M:%S")
    print(fecha_formateada)

    print("Cargar Info")
    info={"description": "Example Dataset", 
      "url": "https://github.com/waspinator/pycococreator", 
      "version": "0.1.0", 
      "year": 2018, 
      "contributor": "waspinator", 
      "date_created": f"{fecha_formateada}"}
    
    print("Cargar Licenses")
    licenses=[{"id": 1, "name": "Attribution-NonCommercial-ShareAlike License", 
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"}]

    print("Cargar Categories")
    categories=[]
    l_categories=LABELS
    for i in range(len(l_categories)):
        categories.append({"id":i,
                        "name":l_categories[i],
                        "supercategory":"N/A"})
        
    print("Cargar images")
    images=[]
    l_paths=list(map(lambda x: get_name_file(x)+".jpg",df["Path"]))
    paths_f=[]
    for p in l_paths:
        
        if("China_Drone" in p):
            paths_f.append("China_Drone/train/images/"+p)
        elif("China_MotorBike" in p):
            paths_f.append("China_MotorBike/train/images/"+p)
        elif("Czech" in p):
            paths_f.append("Czech/train/images/"+p)
        elif("India" in p):
            paths_f.append("India/train/images/"+p)
        elif("Japan" in p):
            paths_f.append("Japan/train/images/"+p)
        elif("Norway" in p):
            paths_f.append("Norway/train/images/"+p)
        elif("United_States" in p):
            paths_f.append("United_States/train/images/"+p)
    for i in range(len(l_paths)):
        images.append({"id": i,
                    "file_name": paths_f[i],
                    "width": 600,
                    "height": 600,
                    "date_captured": ""+str(datetime.now().strftime("%Y-%m-%d %H:%M:%S")),
                    "license": 1,
                    "coco_url": "", 
                    "flickr_url": ""
                    })
    
    print("Cargar Annotations")
    # Annotations
    ides=[i for i in range(len(df))]
    df["Id"]=ides

    annotations=[]

    # Por cada fila

    id_ann=0
    for index_fila in range(len(df)):
        fila = df.iloc[index_fila]

        # Obtengo los valores unitarios: index
        id_fila=fila["Id"]

        # Obtener lista de bbox, labels, TiposDanios
        fila_list_bbox=eval(fila["Bboxes"])
        fila_list_labels=eval(fila["Labels"])
        fila_list_danios=eval(fila["TiposDanios"])

        for id_danio in range(len(fila_list_danios)):
            annotations.append({
                "id": int(id_ann),
                "image_id": int(id_fila),
                "category_id": int(LABEL_MAPEADO[fila_list_labels[id_danio]]),
                "iscrowd": 0,
                "area": 0,
                "bbox": [int(num) for num in fila_list_bbox[id_danio]],
                "segmentation": [[0]]
            })
            id_ann+=1

    custom_train={"info":info,
              "licenses":licenses,
              "categories":categories,
              "images":images,
              "annotations": annotations}
    
    # Nombre del archivo JSON que se va a crear
    if(train):
        filename = save_json+"custom_train.json"
    else:
        filename = save_json+"custom_val.json"

    print(f"Guardar Json: {filename} en {save_json}, esta ruta la debe ingresar en el parametro img_folder")
    # Guardar el diccionario como un archivo .json
    with open(filename, 'w') as file:
        json.dump(custom_train, file)  # El parámetro indent=4 es opcional, solo para que el JSON se vea más legible
    print()
    return custom_train, save_json

r=create_json(X_train,save_json="RDD2022/Img/",train=True)
r=create_json(X_test,save_json="RDD2022/Img/",train=False)

2024-09-05 16:16:08
Cargar Info
Cargar Licenses
Cargar Categories
Cargar images
Cargar Annotations
Guardar Json: RDD2022/Img/custom_train.json en RDD2022/Img/, esta ruta la debe ingresar en el parametro img_folder

2024-09-05 16:16:15
Cargar Info
Cargar Licenses
Cargar Categories
Cargar images
Cargar Annotations
Guardar Json: RDD2022/Img/custom_val.json en RDD2022/Img/, esta ruta la debe ingresar en el parametro img_folder



#### 1.3. Cargado de datos.

Para cargar los datos, se utiliza la clase `AutoFeatureExtractor` de la librería `transformers`, donde a partir de la arquitectura del modelo (en este caso "hustvl/yolos-small"), dispone un generador de formatos que facilita la obtención de los datos a partir de archivos .json siguiendo la estructura mostrada en el **ítem 1.2. de este modelo**.

In [36]:
# Inicializar el extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("hustvl/yolos-small", size=512, max_size=864)

c:\Users\denil\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\yolos\feature_extraction_yolos.py:38: FutureWarning: The class YolosFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use YolosImageProcessor instead.
  warnings.warn(
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Luego, se debe asignar en el parámetro `img_folder` la ruta donde se cargarán los .json que contiene la metadata dispuesta. Para eso, se utiliza la clase CocoDetection(), que tiene preparado el formato establecido anteriormente, y facilita la entrada a los datos de las imágenes.

In [38]:
class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, feature_extractor, train=True):
        ann_file = os.path.join(img_folder, "custom_train.json" if train else "custom_val.json")
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        # read in PIL image and target in COCO format
        img, target = super(CocoDetection, self).__getitem__(idx)
        
        # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.feature_extractor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        target = encoding["labels"][0] # remove batch dimension

        return pixel_values, target

In [39]:
train_dataset = CocoDetection(img_folder=f'RDD2022/Img', feature_extractor=feature_extractor)
val_dataset = CocoDetection(img_folder=f'RDD2022/Img', feature_extractor=feature_extractor, train=False)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [41]:
# Cantidad de datos instanciados con la clase CocoDetection

print("Number of training examples:", len(train_dataset))
print("Number of val examples:", len(val_dataset))

Number of training examples: 14396
Number of val examples: 1600


Con lo anterior, se prepara un dataloader que contendrá los datos dispuestos para los procesos de entrenamiento y pruebas. Utilizando un batch de 3.

In [42]:
def collate_fn(batch):
  pixel_values = [item[0] for item in batch]
  encoding = feature_extractor.pad(pixel_values, return_tensors="pt")
  labels = [item[1] for item in batch]
  batch = {}
  batch['pixel_values'] = encoding['pixel_values']
  batch['labels'] = labels
  return batch

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=3, shuffle=True)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=3)
batch = next(iter(train_dataloader))

### 2. Entrenamiento.

La configuración del modelo para realizar el entrenamiento, es dado a un modulo `Lightning` de la librería `pytorch_lightning`, donde se buscará usar CUDA para realizar un entrenamiento menos demorado. Así que se configura una clase para instanciar la estructura del modelo pre-entrenado y se utilizará el optimizador Adam ponderado.

In [43]:
cats = train_dataset.coco.cats
id2label = {k: v['name'] for k,v in cats.items()}

In [45]:
class Detr(pl.LightningModule):

     def __init__(self, lr, weight_decay):
         super().__init__()
         # replace COCO classification head with custom head
         self.model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-small", 
                                                             num_labels=len(id2label),
                                                             ignore_mismatched_sizes=True)
         # see https://github.com/PyTorchLightning/pytorch-lightning/pull/1896
         self.lr = lr
         self.weight_decay = weight_decay

     def forward(self, pixel_values):
       outputs = self.model(pixel_values=pixel_values)

       return outputs
     
     def common_step(self, batch, batch_idx):
       pixel_values = batch["pixel_values"]
       labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

       outputs = self.model(pixel_values=pixel_values, labels=labels)

       loss = outputs.loss
       loss_dict = outputs.loss_dict

       return loss, loss_dict

     def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)
        for k,v in loss_dict.items():
          self.log("train_" + k, v.item())

        return loss

     def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss)
        for k,v in loss_dict.items():
          self.log("validation_" + k, v.item())

        return loss

     def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr,
                                  weight_decay=self.weight_decay)
        
        return optimizer

     def train_dataloader(self):
        return train_dataloader

     def val_dataloader(self):
        return val_dataloader

In [47]:
model = Detr(lr=2.5e-1, weight_decay=1e-4)

outputs = model(pixel_values=batch['pixel_values'])

Some weights of YolosForObjectDetection were not initialized from the model checkpoint at hustvl/yolos-small and are newly initialized because the shapes did not match:
- class_labels_classifier.layers.2.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([9]) in the model instantiated
- class_labels_classifier.layers.2.weight: found shape torch.Size([92, 384]) in the checkpoint and torch.Size([9, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
trainer = Trainer(gradient_clip_val=0.1, 
                  accumulate_grad_batches=4,
                  max_epochs=20)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### 3. Evaluación del modelo.

Siguiendo las implementaciones anteriores, se podría usar cualquier conjunto de datos para evaluarlo. Usando el conjunto de datos en prueba (val), se obtienen los siguientes resultados.

Se debe realizar un acceso a un repositorio en GitHub para poder realizar la verificación.

```bash
git clone https://github.com/facebookresearch/detr.git
```



In [60]:
%cd detr

from detr.datasets import get_coco_api_from_dataset

base_ds = get_coco_api_from_dataset(val_dataset) # this is actually just calling the coco attribute

%cd ../

In [ ]:
%cd detr

from datasets.coco_eval import CocoEvaluator
from tqdm.notebook import tqdm

iou_types = ['bbox']
coco_evaluator = CocoEvaluator(base_ds, iou_types) # initialize evaluator with ground truths

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()

print("Running evaluation...")

for idx, batch in enumerate(tqdm(val_dataloader)):
    # get the inputs
    pixel_values = batch["pixel_values"].to(device)
    labels = [{k: v.to(device) for k, v in t.items()} for t in batch["labels"]] # these are in DETR format, resized + normalized

    # forward pass
    outputs = model(pixel_values=pixel_values)

    orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    results = feature_extractor.post_process(outputs, orig_target_sizes) # convert outputs of model to COCO api
    res = {target['image_id'].item(): output for target, output in zip(labels, results)}
    coco_evaluator.update(res)

coco_evaluator.synchronize_between_processes()
coco_evaluator.accumulate()
coco_evaluator.summarize()

%cd ../

Con lo anterior, se obtiene los siguientes resultados.

<img src="metricasIoUdataJin.png">

##### Conclusiones.

* Notamos que las metricas obtenidas, no alcanzan a satisfacer las necesidades, en comparación con la del primer modelo.

* En temas computacionales, la duración de entrenamiento de este modelo fue mayor duración por epoc, durando 12 min por epoch, en comparación del modelo v8 con una duración de 8 min.
